# XGBoost Regression Cell Microscopy Model example

This notebook is an example workflow for generating an XGBoost Regression model for scoring 384-well cell plate assays from a randomized plate

## Required Files

- `data_cols` list of desired measurement columns, in this case a python pickle object
- `Per_object.csv` a Cellprofiler measurement file of object level measurements
- `Per_Image.csv` a Cellprofiler measurement file of Image level measurements, we will only be using the `ImageNumber`, `Image_Metadata_WellID` and `Image_Metadata_PlateID` columns for metadata
- `Compounds.csv` a metadata csv containing needed metadata columns: `WellID`, `Compound`
- Known names of the positive and negative controll compounds.

## Step 1: Imports

Imports for generating a scored mode

- `os` and `re` Operating system for interacting with file paths and regex for pattern matching in strings
- `numpy` and `pandas` tabular dataframes and interacting with the data
- `sklearn` plate based normalization, train/validation split and model quality metrics
- `xgb` Gradient boosted trees, the source for the model
- `shap` Shapley analysis for feature importance
- `matplotlib` and `seaborn` graphing utilities
- `glob` for finding lists of file in a directory (needed for multiple plate models)
- `cellutils.plot` plotting function for well level plates to include hits that are (6*std)+mean the compounds
- `cellutils.utils` helper functions for reducing object level to well level means, generating a character range (e.g. A01->A24) for control lists, generating data cols based on typical measurement names and zprime for assay quality
- `sqlite3` python sqlite 3 engine for using source Cellprofiler outputs
- `pickle` package for saving/loading python objects in this case `data_cols`

In [ ]:
import os
import re

import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, r2_score, mean_squared_error

import xgboost as xgb
import shap

import matplotlib.pyplot as plt
import seaborn as sns

import glob 

from cellutils.plot import plot_plate
from cellutils.utils import make_well, char_range, get_data_cols, zprime

import sqlite3
import pickle

#######################################
### Change these paths to your data ###
data_cols_path = 'path/to/data_cols'
obj_path = 'path/to/object/file.csv'
img_path = 'path/to/image/file.csv'
meta_path  = 'path/to/meta/file.csv'
#######################################

with open(data_cols_path, 'rb') as f:
    data_cols = pickle.load(f)

## Loading Data, Joining data, Preparing data

We will read in the object data in chunks to save on memory during the read in

- Read in all of the Data
- Join metadata to data
- Add `label` and `Condition` for later use

Run both cells, the `head()` and `shape` prints at the end of the first is for sanity checking. The number of rows should not change but columns should

In [ ]:
# Data read/join
df = []
for chunk in pd.read_csv(obj_path, chunksize=int(5e5)):
    print(f"Read {len(chunk)} rows...")
    df.append(chunk)
del chunk # probably unneeded but just in case of memory issues
df = pd.concat(df)
meta = pd.read_csv(img_path, usecols=['ImageNumber', 'Image_Metadata_WellID', 'Image_Metadata_PlateID'])
cmpds = pd.read_csv(meta_path)
meta = pd.merge(meta, cmpds, left_on='Image_Metadata_WellID', right_on='WellID') # common issue in differing well_id column names, change as needed
del cmpds # again memory issues, maybe

print(df.shapel, meta.shape)
df = pd.merge(df, meta, on='Image_Metadata_WellID', how='left') # the left is implied but occasionally this is needed
print(df.shapel, meta.shape)

df.head()

In [ ]:
# Data prepare
#######################################
####### Control Compound Names ########
pc_name = 'good_drug'
nc_name = 'no_drug_DMSO'
#######################################

# Condition column for plot coloring
df['condition'] = 'cmpd'
df.loc[df['compound']==pc_name, 'condition'] = 'pc'
df.loc[df['compound']==nc_name, 'condition'] = 'nc'

# dt will be our control data
dt = df.loc[df['condition']!='cmpd']

# label column for data training
dt['label'] = 0
dt.loc[dt['cond'] =='PC', 'label'] = 1
dt.head()

## Creating and training the Model

- Define an X and Y for the model (scaled measurements and label)
- Divide data into train and validate sets
- Train the model and get quality metrics

`evalset` is used for generating a loss curve (AUC, ROC)

In [ ]:
y = dt['label'].values
X = StandardScaler().fit_transform(dt[data_cols])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)
evalset = [(X_train, y_train), (X_test,y_test)] 

In [ ]:
model = xgb.XGBRegressor()
model.fit(X_train, y_train, eval_metric='logloss', eval_set=evalset)

## Model Quality Metrics

We have the model predict a label on the reserved validation (`test`) set to compare against naive data it hasn't been trained on

### Metrics
- Accuracy (score is rounded to 1 or 0 and compared to validation set)
- R^2 (higher better nerver reaching 1)
- Mean Squared Error (MSE) lower is better

In [ ]:
y_pred = model.predict(X_test)
acc_preds = [round(val) for val in y_pred]
ac = accuracy_score(y_test, acc_preds)

r2 = r2_score(y_pred, y_test)
mse = mean_squared_error(y_pred, y_test)

print("IR500")
print(f"Accuracy: {ac*100}%")
print(f"R2:{r2:.3f}")
print(f"MSE:{mse:.3f}")

### Loss/AUC/ROC

Change names as needed

In [ ]:
results = model.evals_result()
plt.plot(results['validation_0']['logloss'], label='train')
plt.plot(results['validation_1']['logloss'], label='test')

plt.legend()

plt.title("IR500 AUC")
plt.savefig("plots/auc_ir500.svg", format='SVG', bbox_inches="tight")

## SHAP Analysis

Shap is based on Shapely analysis is for which features are providing more to the aggregate of all features

In [ ]:
explainer = shap.TreeExplainer(model)
shap_values = explainer(X)
shap.summary_plot(shap_values, X, feature_names=data_cols, show=False,)
plt.savefig('plots/shap_ir500.png', format='png', bbox_inches='tight')

## Z-Prime

Have the model predict the whole control set and calculate zprime

In [ ]:
dt['score'] = model.predict(X)
pcs = dt.loc[dt['cond']=='PC', 'score']
ncs = dt.loc[dt['cond']=='NC', 'score']
zp = zprime(pcs, ncs)
print(f"Z-prime: {zp:.3f}")

In [ ]:
# Save the model for future use
model.save_model('models/ir500_model')